# Laporan Praktikum 2 – Kecerdasan Komputasional  
## Proof by Resolution, Forward & Backward Chaining, dan First-Order Logic  


### Identitas Mahasiswa
- Nama  : Wayan Raditya Putra  
- NRP   : 5054241029  
- Program Studi : Rekayasa Kecerdasan Artifisial
- Angkatan : 2024  


### Tujuan Praktikum
Praktikum ini bertujuan untuk:  
1. Memahami konsep dasar Proof by Resolution sebagai teknik inferensi dalam logika.  
2. Mempelajari mekanisme Forward Chaining dan Backward Chaining untuk penalaran berbasis aturan.  
3. Mengenal representasi First-Order Logic (FOL) dan bagaimana FOL digunakan dalam basis pengetahuan.  
4. Mengimplementasikan berbagai teknik inferensi untuk menyelesaikan permasalahan berbasis logika komputasional.  


### Deskripsi Singkat
Pada praktikum kedua ini, mahasiswa diminta untuk:  
- Menyusun representasi basis pengetahuan dalam bentuk proposisi maupun logika orde pertama.  
- Menerapkan teknik inferensi seperti resolution, forward chaining, backward chaining.  
- Melakukan percobaan terhadap kasus sederhana untuk melihat bagaimana sistem inferensi menghasilkan kesimpulan dari basis pengetahuan.  


## Instalasi dan Import Library

**Tujuan:**  
Menyiapkan library dan modul yang diperlukan sebelum menjalankan praktikum. Pada tahap ini dilakukan instalasi paket eksternal serta import fungsi-fungsi yang sudah tersedia di file pendukung.

```python
%pip install ipythonblocks
%pip install qpsolvers

from utils import *
from logic import *
from notebook import psource
````
**Penjelasan:**

* `%pip install ipythonblocks` → menginstal library **ipythonblocks** yang digunakan untuk menampilkan blok warna pada notebook.
* `%pip install qpsolvers` → menginstal library **qpsolvers** yang digunakan untuk menyelesaikan masalah Quadratic Programming.
* `from utils import *` → mengimpor seluruh fungsi dari file `utils.py` yang berisi kumpulan fungsi bantu.
* `from logic import *` → mengimpor seluruh fungsi dari file `logic.py` yang berisi implementasi logika proposisional dan inference.
* `from notebook import psource` → mengimpor fungsi `psource` dari `notebook.py`, yang digunakan untuk menampilkan source code suatu fungsi langsung di notebook.



In [1]:
%pip install ipythonblocks
%pip install qpsolvers
from utils import *
from logic import *
from notebook import psource

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


d:\AI_Journey\Computational_Intelligence\CI\Lib\site-packages\qpsolvers\solvers\__init__.py:880: UserWarning: no QP solver found on your system, you can install solvers from PyPI by ``pip install qpsolvers[open_source_solvers]``
  warnings.warn(


## Inferensi pada Basis Pengetahuan Proposisional

**Tujuan:**  
Pada bagian ini kita akan mempelajari dua teknik utama yang digunakan untuk memeriksa apakah suatu kalimat (sentence) dapat di-*entail* oleh sebuah basis pengetahuan (`KB`).  






### Proof by Resolution

Tujuan kita adalah memeriksa apakah $KB \vDash \alpha$, yaitu apakah $KB \implies \alpha$ bernilai benar di setiap model.  

Contoh: kita ingin memeriksa apakah $P \implies Q$ valid.  
Untuk itu kita periksa keterpuasan dari $\neg (P \implies Q)$, yang bisa ditulis ulang menjadi:  

$$
P \land \neg Q
$$

Jika ekspresi $P \land \neg Q$ **tidak terpuaskan (unsatisfiable)**, maka $P \implies Q$ pasti benar di semua model.  


### Konsep Dasar

Teknik ini dikenal sebagai **pembuktian dengan kontradiksi** (*proof by contradiction*).  
Langkahnya: kita mengasumsikan bahwa $\alpha$ salah, lalu menunjukkan bahwa asumsi ini menimbulkan kontradiksi dengan fakta-fakta dalam $KB$.  


### Aturan Resolusi

Aturan resolusi berbunyi:  

$$
(l_1 \lor \dots \lor l_k), \quad (m_1 \lor \dots \lor m_n), \quad (l_i \equiv \neg m_j) 
\;\;\;\Rightarrow\;\;\;
(l_1 \lor \dots \lor l_{i-1} \lor l_{i+1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j-1} \lor m_{j+1} \lor \dots \lor m_n)
$$


### Hasil Resolusi

Proses resolusi dijalankan berulang sampai salah satu kondisi berikut:

1. Tidak ada klausa baru yang bisa ditambahkan → artinya $KB \nvDash \alpha$.  
2. Ditemukan **klausa kosong** → artinya $KB \vDash \alpha$.  

Klausa kosong setara dengan **False**, karena hanya bisa muncul dari resolusi dua klausa saling berlawanan, misalnya $P$ dan $\neg P$.


# Konversi Kalimat ke Bentuk Normal Konjungtif (CNF)

Dalam pembuktian dengan **resolusi**, algoritma tidak bisa langsung menangani kalimat logika yang rumit.  
Karena itu, kalimat yang mengandung **implikasi (→)** dan **bi-implikasi (↔)** harus disederhanakan dulu.

👉 Fakta penting: **setiap kalimat logika proposisional setara dengan bentuk konjungsi dari klausa.**  
Bentuk ini disebut **Conjunctive Normal Form (CNF)**, yaitu **konjungsi (AND)** dari beberapa **disjungsi (OR) literal**.

Contoh CNF:

$$
(A \lor B) \land (\neg B \lor C \lor \neg D) \land (D \lor \neg E)
$$

Bentuk ini sama seperti **Product of Sums (POS)** di elektronika digital.


## Tahapan Konversi ke CNF

### 1. Ubah Bi-Implikasi ke Implikasi
$$
\alpha \iff \beta \equiv (\alpha \implies \beta) \land (\beta \implies \alpha)
$$

Jika bagian kanan berupa kalimat majemuk:
$$
\alpha \iff (\beta \lor \gamma) \equiv (\alpha \implies (\beta \lor \gamma)) \land ((\beta \lor \gamma) \implies \alpha)
$$


### 2. Ubah Implikasi ke Bentuk Setara
$$
\alpha \implies \beta \equiv \neg \alpha \lor \beta
$$


### 3. Pindahkan Negasi ke Literal
Negasi hanya boleh menempel pada proposisi atomik.  
Gunakan **Hukum De Morgan**:

$$
\neg(\alpha \land \beta) \equiv (\neg \alpha \lor \neg \beta)
$$

$$
\neg(\alpha \lor \beta) \equiv (\neg \alpha \land \neg \beta)
$$


### 4. Distribusikan Disjungsi terhadap Konjungsi
Agar sesuai format CNF, lakukan distribusi:

$$
(\alpha \lor (\beta \land \gamma)) \equiv (\alpha \lor \beta) \land (\alpha \lor \gamma)
$$

Bentuk akhir yang kita inginkan:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land (\gamma_1 \lor \gamma_2 \lor \dots)
$$


## Inti Proses
- CNF = **“AND of ORs”** (konjungsi dari disjungsi literal).  
- Proses konversi = hilangkan ↔ dan →, dorong negasi ke literal, lalu distribusikan OR atas AND.  
- Dengan CNF, kalimat siap dipakai dalam algoritma **resolusi**.




In [2]:
psource(to_cnf)

## Fungsi `to_cnf`
Untuk mengubah sebuah kalimat logika proposisional ke bentuk **Conjunctive Normal Form (CNF)**, kita gunakan fungsi `to_cnf`.  

```python
def to_cnf(s): 
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)        # Step 3
    return distribute_and_over_or(s)  # Step 4
```

### Penjelasan Tahapan

1. **`eliminate_implications(s)`**

   * Menyelesaikan **Langkah 1 dan 2** dari proses konversi.
   * Semua bi-implikasi (\$\alpha \iff \beta\$) diubah menjadi dua implikasi.
   * Semua implikasi (\$\alpha \implies \beta\$) diubah menjadi bentuk setara (\$\neg \alpha \lor \beta\$).

2. **`move_not_inwards(s)`**

   * Menyelesaikan **Langkah 3**.
   * Negasi digeser ke dalam agar hanya menempel pada literal, dengan bantuan hukum De Morgan.
   * Contoh:

     $$
     \neg (A \lor B) \equiv (\neg A \land \neg B)
     $$

3. **`distribute_and_over_or(s)`**

   * Menyelesaikan **Langkah 4**.
   * Melakukan distribusi disjungsi terhadap konjungsi, sehingga hasil akhir berbentuk **konjungsi dari disjungsi literal**.
   * Contoh:

     $$
     (A \lor (B \land C)) \equiv (A \lor B) \land (A \lor C)
     $$


### Intinya

* Fungsi `to_cnf` = implementasi praktis dari 4 langkah konversi CNF.
* Setiap kalimat proposisional bisa dipaksa ke CNF → siap dipakai untuk **algoritma resolusi**.



In [3]:
psource(eliminate_implications)
psource(move_not_inwards)
psource(distribute_and_over_or)


## Fungsi Pendukung `to_cnf`

Agar `to_cnf` bekerja, ada tiga fungsi utama yang menjalankan proses sesuai 4 langkah konversi CNF:

### 1. `eliminate_implications(s)`

```python
def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)
```

**Fungsi:**

* Menghilangkan semua **implikasi (⇒)** dan **bi-implikasi (⇔)**.
* Hasil akhirnya hanya mengandung operator `&` (AND), `|` (OR), dan `~` (NOT).
* Contoh:

$$
(P \implies Q) \equiv (\neg P \lor Q)
$$

$$
(P \iff Q) \equiv (P \implies Q) \land (Q \implies P)
$$


### 2. `move_not_inwards(s)`

```python
def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
        if a.op == '&':
            return associate('|', list(map(NOT, a.args)))
        if a.op == '|':
            return associate('&', list(map(NOT, a.args)))
        return s
    elif is_symbol(s.op) or not s.args:
        return s
    else:
        return Expr(s.op, *list(map(move_not_inwards, s.args)))
```

**Fungsi:**

* Memindahkan **negasi** supaya hanya menempel pada literal (variabel atomik).
* Menggunakan **Hukum De Morgan** dan penyederhanaan ganda:

$$
\neg (A \lor B) \equiv (\neg A \land \neg B)
$$

$$
\neg (A \land B) \equiv (\neg A \lor \neg B)
$$

$$
\neg(\neg A) \equiv A
$$


### 3. `distribute_and_over_or(s)`

```python
def distribute_and_over_or(s):
    """Given a sentence s consisting of conjunctions and disjunctions
    of literals, return an equivalent sentence in CNF.
    >>> distribute_and_over_or((A & B) | C)
    ((A | C) & (B | C))
    """
    s = expr(s)
    if s.op == '|':
        s = associate('|', s.args)
        if s.op != '|':
            return distribute_and_over_or(s)
        if len(s.args) == 0:
            return False
        if len(s.args) == 1:
            return distribute_and_over_or(s.args[0])
        conj = first(arg for arg in s.args if arg.op == '&')
        if not conj:
            return s
        others = [a for a in s.args if a is not conj]
        rest = associate('|', others)
        return associate('&', [distribute_and_over_or(c | rest)
                               for c in conj.args])
    elif s.op == '&':
        return associate('&', list(map(distribute_and_over_or, s.args)))
    else:
        return s
```

**Fungsi:**

* Melakukan distribusi **OR terhadap AND**, supaya hasil akhir sesuai format CNF.
* Contoh:

$$
(A \land B) \lor C \equiv (A \lor C) \land (B \lor C)
$$


## Alur Lengkap

Jadi, `to_cnf` menjalankan ketiga fungsi ini berurutan:

1. **Hilangkan implikasi** → `eliminate_implications(s)`
2. **Pindahkan negasi** → `move_not_inwards(s)`
3. **Distribusikan OR atas AND** → `distribute_and_over_or(s)`

Hasil akhirnya selalu berbentuk:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land \dots
$$

siap dipakai dalam algoritma **resolusi**.




Mari kita coba untuk konversi beberapa kalimat menjadi cnf

In [4]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [5]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [6]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [7]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

Coming back to our resolution problem, we can see how the `to_cnf` function is utilized here

In [8]:
psource(pl_resolution)

In [9]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

In [10]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

(False, False)

### Forward and backward chaining
Previously, we said we will look at two algorithms to check if a sentence is entailed by the `KB`. Here's a third one. 
The difference here is that our goal now is to determine if a knowledge base of definite clauses entails a single proposition symbol *q* - the query.
There is a catch however - the knowledge base can only contain **Horn clauses**.
<br>
#### Horn Clauses
Horn clauses can be defined as a *disjunction* of *literals* with **at most** one positive literal. 
<br>
A Horn clause with exactly one positive literal is called a *definite clause*.
<br>
A Horn clause might look like 
<br>
$\neg a\lor\neg b\lor\neg c\lor\neg d... \lor z$
<br>
This, coincidentally, is also a definite clause.
<br>
Using De Morgan's laws, the example above can be simplified to 
<br>
$a\land b\land c\land d ... \implies z$
<br>
This seems like a logical representation of how humans process known data and facts. 
Assuming percepts `a`, `b`, `c`, `d` ... to be true simultaneously, we can infer `z` to also be true at that point in time. 
There are some interesting aspects of Horn clauses that make algorithmic inference or *resolution* easier.
- Definite clauses can be written as implications:
<br>
The most important simplification a definite clause provides is that it can be written as an implication.
The premise (or the knowledge that leads to the implication) is a conjunction of positive literals.
The conclusion (the implied statement) is also a positive literal.
The sentence thus becomes easier to understand.
The premise and the conclusion are conventionally called the *body* and the *head* respectively.
A single positive literal is called a *fact*.
- Forward chaining and backward chaining can be used for inference from Horn clauses:
<br>
Forward chaining is semantically identical to `AND-OR-Graph-Search` from the chapter on search algorithms.
Implementational details will be explained shortly.
- Deciding entailment with Horn clauses is linear in size of the knowledge base:
<br>
Surprisingly, the forward and backward chaining algorithms traverse each element of the knowledge base at most once, greatly simplifying the problem.
<br>
<br>
The function `pl_fc_entails` implements forward chaining to see if a knowledge base `KB` entails a symbol `q`.
<br>
Before we proceed further, note that `pl_fc_entails` doesn't use an ordinary `KB` instance. 
The knowledge base here is an instance of the `PropDefiniteKB` class, derived from the `PropKB` class, 
but modified to store definite clauses.
<br>
The main point of difference arises in the inclusion of a helper method to `PropDefiniteKB` that returns a list of clauses in KB that have a given symbol `p` in their premise.

In [11]:
psource(PropDefiniteKB.clauses_with_premise)

Let's now have a look at the `pl_fc_entails` algorithm.

In [12]:
psource(pl_fc_entails)

The function accepts a knowledge base `KB` (an instance of `PropDefiniteKB`) and a query `q` as inputs.
<br>
<br>
`count` initially stores the number of symbols in the premise of each sentence in the knowledge base.
<br>
The `conjuncts` helper function separates a given sentence at conjunctions.
<br>
`inferred` is initialized as a *boolean* defaultdict. 
This will be used later to check if we have inferred all premises of each clause of the agenda.
<br>
`agenda` initially stores a list of clauses that the knowledge base knows to be true.
The `is_prop_symbol` helper function checks if the given symbol is a valid propositional logic symbol.
<br>
<br>
We now iterate through `agenda`, popping a symbol `p` on each iteration.
If the query `q` is the same as `p`, we know that entailment holds.
<br>
The agenda is processed, reducing `count` by one for each implication with a premise `p`.
A conclusion is added to the agenda when `count` reaches zero. This means we know all the premises of that particular implication to be true.
<br>
`clauses_with_premise` is a helpful method of the `PropKB` class.
It returns a list of clauses in the knowledge base that have `p` in their premise.
<br>
<br>
Now that we have an idea of how this function works, let's see a few examples of its usage, but we first need to define our knowledge base. We assume we know the following clauses to be true.

In [13]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

We will now `tell` this information to our knowledge base.

In [14]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

We can now check if our knowledge base entails the following queries.

In [15]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

In [16]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [17]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [18]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

## First-Order Logic Knowledge Bases: `FolKB`

The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. You would initialize and use it the same way as you would for `PropKB` except that the clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

## Criminal KB
In this section we create a `FolKB` based on the following paragraph.<br/>
<em>The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.</em><br/>
The first step is to extract the facts and convert them into first-order definite clauses. Extracting the facts from data alone is a challenging task. Fortunately, we have a small paragraph and can do extraction and conversion manually. We'll store the clauses in list aptly named `clauses`.

In [19]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>
The keywords to look for here are 'crime', 'American', 'sell', 'weapon' and 'hostile'. We use predicate symbols to make meaning of them.

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Let us now combine them with appropriate variable naming to depict the meaning of the sentence. The criminal `x` is also the American `x` who sells weapon `y` to `z`, which is a hostile nation.

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [20]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"The country Nono, an enemy of America"</em><br/>
We now know that Nono is an enemy of America. We represent these nations using the constant symbols `Nono` and `America`. the enemy relation is show using the predicate symbol `Enemy`.

$\text{Enemy}(\text{Nono}, \text{America})$

In [21]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [22]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>
If Nono owns something and it classifies as a missile, then it was sold to Nono by West.

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [23]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>
West is an American.

$\text{American}(\text{West})$

In [24]:
clauses.append(expr("American(West)"))

We also know, from our understanding of language, that missiles are weapons and that an enemy of America counts as “hostile”.

$\text{Missile}(x) \implies \text{Weapon}(x), \text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [25]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [26]:
crime_kb = FolKB(clauses)

The `subst` helper function substitutes variables with given values in first-order logic statements.
This will be useful in later algorithms.
It's implementation is quite simple and self-explanatory.

In [27]:
psource(subst)

Here's an example of how `subst` can be used.

In [28]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

## Inference in First-Order Logic
In this section we look at a forward chaining and a backward chaining algorithm for `FolKB`. Both aforementioned algorithms rely on a process called <strong>unification</strong>, a key component of all first-order inference algorithms.

### Unification
We sometimes require finding substitutions that make different logical expressions look identical. This process, called unification, is done by the `unify` algorithm. It takes as input two sentences and returns a <em>unifier</em> for them if one exists. A unifier is a dictionary which stores the substitutions required to make the two sentences identical. It does so by recursively unifying the components of a sentence, where the unification of a variable symbol `var` with a constant symbol `Const` is the mapping `{var: Const}`. Let's look at a few examples.

In [29]:
unify(expr('x'), 3)

{x: 3}

In [30]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [31]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

In cases where there is no possible substitution that unifies the two sentences the function return `None`.

In [32]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

None


We also need to take care we do not unintentionally use the same variable name. Unify treats them as a single variable which prevents it from taking multiple value.

In [33]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None


### Forward Chaining Algorithm
We consider the simple forward-chaining algorithm presented in <em>Figure 9.3</em>. We look at each rule in the knowledge base and see if the premises can be satisfied. This is done by finding a substitution which unifies each of the premise with a clause in the `KB`. If we are able to unify the premises, the conclusion (with the corresponding substitution) is added to the `KB`. This inferencing process is repeated until either the query can be answered or till no new sentences can be added. We test if the newly added clause unifies with the query in which case the substitution yielded by `unify` is an answer to the query. If we run out of sentences to infer, this means the query was a failure.

The function `fol_fc_ask` is a generator which yields all substitutions which validate the query.

In [34]:
psource(fol_fc_ask)

Let's find out all the hostile nations. Note that we only told the `KB` that Nono was an enemy of America, not that it was hostile.

In [35]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


The generator returned a single substitution which says that Nono is a hostile nation. See how after adding another enemy nation the generator returns two substitutions.

In [36]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


<strong><em>Note</em>:</strong> `fol_fc_ask` makes changes to the `KB` by adding sentences to it.

### Backward Chaining Algorithm
This algorithm works backward from the goal, chaining through rules to find known facts that support the proof. Suppose `goal` is the query we want to find the substitution for. We find rules of the form $\text{lhs} \implies \text{goal}$ in the `KB` and try to prove `lhs`. There may be multiple clauses in the `KB` which give multiple `lhs`. It is sufficient to prove only one of these. But to prove a `lhs` all the conjuncts in the `lhs` of the clause must be proved. This makes it similar to <em>And/Or</em> search.

#### OR
The <em>OR</em> part of the algorithm comes from our choice to select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules's `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [37]:
psource(fol_bc_or)

#### AND
The <em>AND</em> corresponds to proving all the conjuncts in the `lhs`. We need to find a substitution which proves each <em>and</em> every clause in the list of conjuncts.

In [38]:
psource(fol_bc_and)

Now the main function `fl_bc_ask` calls `fol_bc_or` with substitution initialized as empty. The `ask` method of `FolKB` uses `fol_bc_ask` and fetches the first substitution returned by the generator to answer query. Let's query the knowledge base we created from `clauses` to find hostile nations.

In [39]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [40]:
crime_kb.ask(expr('Hostile(x)'))

{v_5: Nono, x: Nono}

You may notice some new variables in the substitution. They are introduced to standardize the variable names to prevent naming problems as discussed in the [Unification section](#Unification)

## Appendix: The Implementation of `|'==>'|`

Consider the `Expr` formed by this syntax:

In [41]:
P |'==>'| ~Q

(P ==> ~Q)

What is the funny `|'==>'|` syntax? The trick is that "`|`" is just the regular Python or-operator, and so is exactly equivalent to this: 

In [42]:
(P | '==>') | ~Q

(P ==> ~Q)

In other words, there are two applications of or-operators. Here's the first one:

In [43]:
P | '==>'

PartialExpr('==>', P)

What is going on here is that the `__or__` method of `Expr` serves a dual purpose. If the right-hand-side is another `Expr` (or a number), then the result is an `Expr`, as in `(P | Q)`. But if the right-hand-side is a string, then the string is taken to be an operator, and we create a node in the abstract syntax tree corresponding to a partially-filled  `Expr`, one where we know the left-hand-side is `P` and the operator is `==>`, but we don't yet know the right-hand-side.

The `PartialExpr` class has an `__or__` method that says to create an `Expr` node with the right-hand-side filled in. Here we can see the combination of the `PartialExpr` with `Q` to create a complete `Expr`:

In [44]:
partial = PartialExpr('==>', P) 
partial | ~Q

(P ==> ~Q)

This  [trick](http://code.activestate.com/recipes/384122-infix-operators/) is due to [Ferdinand Jamitzky](http://code.activestate.com/recipes/users/98863/), with a modification by [C. G. Vedant](https://github.com/Chipe1),
who suggested using a string inside the or-bars.

## Appendix: The Implementation of `expr`

How does `expr` parse a string into an `Expr`? It turns out there are two tricks (besides the Jamitzky/Vedant trick):

1. We do a string substitution, replacing "`==>`" with "`|'==>'|`" (and likewise for other operators).
2. We `eval` the resulting string in an environment in which every identifier
is bound to a symbol with that identifier as the `op`.

In other words,

In [45]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

is equivalent to doing:

In [46]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

One thing to beware of: this puts `==>` at the same precedence level as `"|"`, which is not quite right. For example, we get this:

In [47]:
P & Q  |'==>'|  P | Q

(((P & Q) ==> P) | Q)

which is probably not what we meant; when in doubt, put in extra parens:

In [48]:
(P & Q)  |'==>'|  (P | Q)

((P & Q) ==> (P | Q))

## Examples

In [49]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(x)'))

# Authors

This notebook by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).

